In [ ]:

# training with torchtune notes...

import torch
from torchtune import TuneConfig
from torchtune.models.llama import create_model
from transformers import AutoTokenizer

model_name = "meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the base Llama model in Torchtune
model = create_model(model_name, quantization="bnb_4bit")  # Load model with 4-bit quantization

from datasets import load_dataset

# Load dataset from JSONL file
dataset = load_dataset("json", data_files="fine_tuning_data.jsonl", split="train")

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["input"], padding="max_length", truncation=True, max_length=512)

# Apply tokenization
dataset = dataset.map(tokenize_function, batched=True)
dataset = dataset.rename_column("expected", "labels")  # Rename correct output to labels

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

from torchtune.trainers import Trainer

# Define fine-tuning config
train_config = TuneConfig(
    output_dir="./fine_tuned_llama",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    save_strategy="epoch",
    fp16=True
)

trainer = Trainer(
    model=model,
    config=train_config,
    train_dataset=dataset
)

# Start fine-tuning
trainer.train()

model.save_pretrained("./fine_tuned_llama")
tokenizer.save_pretrained("./fine_tuned_llama")


In [ ]:

# eval

from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline

fine_tuned_model_name = "./fine_tuned_llama"

# Load fine-tuned model for inference
pipe = pipeline("text-generation", model=fine_tuned_model_name, tokenizer=tokenizer, device=0)

# Integrate with LangChain
llm = HuggingFacePipeline(pipeline=pipe)

# Test inference with fine-tuned model
query = "What is the capital of France?"
response = llm.invoke(query)
print(response)


In [ ]:

# SFT data format: ChatML or Alpaca-style data format, save in .jsonl file

{"messages": [
    {"role": "system", "content": "You are a robotic 3D scene graph planning expert. Generate a valid plan using the given JSON scene graph."},
    {"role": "user", "content": "3D Scene Graph: {scene_graph_here} Task: {task_description}"},
    {"role": "assistant", "content": "[\"goto(r_10)\", \"goto(r_6)\", \"pickup(o_9)\", \"goto(r_9)\", \"release(o_9)\"]"}
]}







In [ ]:

# reference message:

{
    "messages": [
        {"role": "system", "content": "{planner_prompt_alt}"},
        {"role": "user", "content": "The room adjacencies to strictly follow:"},
        {"role": "user", "content": "{room_graph}"},
        {"role": "user", "content": "The 3D Scene Graph in JSON format:"},
        {"role": "user", "content": "{json_graph}"},
        {"role": "user", "content": "The task description:"},
        {"role": "user", "content": "{task_description}"},
        {"role": "assistant", "content": "{plan}"}
    ]
}



In [ ]:

import os
import json


directory = '/home/laszlo/Stanford/3dscenegraph/tiny/verified_graph'
json_file = '3DSceneGraph_Allensville_single.json'

file_path = os.path.join(directory, json_file)

with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)

print(len(data["valid"]))

valid_data = data["valid"][0]

scene_graph = valid_data["scene_graph"]
prunded_scene_graph = valid_data["pruned_graph"]
system_prompt = valid_data["system_prompt"]
room_graph = valid_data["room_graph"]
task_description = valid_data["instruction"]
verifying_step = valid_data["validation_steps"]
plan = valid_data["plan"]



In [ ]:

#print(scene_graph)
#print(prunded_scene_graph)
#print(system_prompt)
#print(room_graph)
#print(task_description)
#print(verifying_step)
print(plan)

# converting list to json string
plan_json = json.dumps(plan)
print(plan_json.type)




In [ ]:
# SFT data loader

import os
import json

from tqdm import tqdm


def sft_training_data_loader(directory):
    
    examples = []

    # Get list of JSON files in the directory
    json_files = [f for f in os.listdir(directory) if f.endswith(".json")]

    print(f"🔍 Found {len(json_files)} JSON files in {directory}")

    if not json_files:
        print(f"No JSON files found in {directory}")
        return

    # Iterate over JSON files and yield parsed content
    for json_file in tqdm(json_files , desc="Processing JSON files", total=len(json_files)):
        file_path = os.path.join(directory, json_file)

        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        for valid_data in data["valid"]:

            scene_graph = valid_data["scene_graph"]
            prunded_scene_graph = valid_data["pruned_graph"]
            system_prompt = valid_data["system_prompt"]
            room_graph = valid_data["room_graph"]
            task_description = valid_data["instruction"]
            verifying_step = valid_data["validation_steps"]
            plan = json.dumps(valid_data["plan"])

            examples.append({
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": f"The room adjacencies to strictly follow: {room_graph} The 3D Scene Graph in JSON format: {prunded_scene_graph} The task description: {task_description}"},
                    {"role": "assistant", "content": f"{plan}"}
                ]
            })

    # 🚀 Write the entire list to JSONL format

    save_path = os.path.join(directory, "sft_trainer_dataset.jsonl")

    with open(save_path, "w", encoding="utf-8") as f:
        for example in examples:
            f.write(json.dumps(example) + "\n")  # Write each JSON object on a new line

    print("JSONL file saved successfully!")

    return examples


In [ ]:

#datagen_output_directory = '/home/laszlo/Stanford/3dscenegraph/datagen_output_tiny'
datagen_output_directory = '/home/laszlo/Stanford/3dscenegraph/datagen_output_merged'


training_examples = sft_training_data_loader(datagen_output_directory)


In [ ]:

# stats data loader

import os
import json

from tqdm import tqdm


def stats_data_loader(directory):
    
    examples = []

    # Get list of JSON files in the directory
    json_files = [f for f in os.listdir(directory) if f.endswith(".json")]

    print(f"🔍 Found {len(json_files)} JSON files in {directory}")

    if not json_files:
        print(f"No JSON files found in {directory}")
        return

    valid_plans = 0
    invalid_plans = 0
    total_plans = 0
    exec_time = 0
    length = 0

    # Iterate over JSON files and yield parsed content
    for json_file in tqdm(json_files , desc="Processing JSON files", total=len(json_files)):
        file_path = os.path.join(directory, json_file)

        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        valid_plans += data["stats"]["valid_plans"]
        invalid_plans += data["stats"]["invalid_plans"]
        total_plans += data["stats"]["total_plans"]
        exec_time += data["stats"]["exec_time"]


        for valid_data in data["valid"]:

            length += len(valid_data["plan"])




    print(f"Valid plans: {valid_plans}")
    print(f"Invalid plans: {invalid_plans}")
    print(f"Total plans: {total_plans}")
    print(f"Execution time sum: {exec_time}")
    print(f"Average execution time: {exec_time/(valid_plans + invalid_plans)}")
    print(f"Average plan length: {length/valid_plans}")

    




In [ ]:

#datagen_output_directory = '/home/laszlo/Stanford/3dscenegraph/datagen_output_tiny'
datagen_output_directory = '/home/laszlo/Stanford/3dscenegraph/datagen_output_merged'


training_examples = stats_data_loader(datagen_output_directory)





In [ ]:
# PPO data loader

import os
import json

from tqdm import tqdm


def ppo_training_data_loader(directory):
    
    examples = []

    # Get list of JSON files in the directory
    json_files = [f for f in os.listdir(directory) if f.endswith(".json")]

    print(f"🔍 Found {len(json_files)} JSON files in {directory}")

    if not json_files:
        print(f" No JSON files found in {directory}")
        return

    # Iterate over JSON files and yield parsed content
    for json_file in tqdm(json_files , desc="Processing JSON files", total=len(json_files)):
        file_path = os.path.join(directory, json_file)

        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        for valid_data in data["valid"]:

            scene_graph = valid_data["scene_graph"]
            prunded_scene_graph = valid_data["pruned_graph"]
            system_prompt = valid_data["system_prompt"]
            room_graph = valid_data["room_graph"]
            task_description = valid_data["instruction"]
            verifying_step = valid_data["validation_steps"]
            plan = json.dumps(valid_data["plan"])

            examples.append({
                "system_prompt": system_prompt,
                "room_graph": room_graph,
                "scene_graph": prunded_scene_graph,
                "task_description": task_description,
                "verifying_step": verifying_step,
                "plan": plan
            })

    # 🚀 Write the entire list to JSONL format

    save_path = os.path.join(directory, "ppo_trainer_dataset.jsonl")

    with open(save_path, "w", encoding="utf-8") as f:
        for example in examples:
            f.write(json.dumps(example) + "\n")  # Write each JSON object on a new line

    print("JSONL file saved successfully!")

    return examples





In [ ]:
datagen_output_directory = '/home/laszlo/Stanford/3dscenegraph/datagen_output_merged'


training_examples = ppo_training_data_loader(datagen_output_directory)

In [ ]:
# toke counter

from transformers import AutoTokenizer


# **1️Load Model & Tokenizer**
model_name = "/home/laszlo/Stanford/LangSpace/meta-llama/Llama-3.2-3B"  # Change this to 1B, 3B, or 8B
tokenizer = AutoTokenizer.from_pretrained(model_name)

def get_example_token_count(example):
    """
    Tokenizes a dataset example and returns the token count.
    """
    full_text = ""
    for message in example["messages"]:
        full_text += message["content"] + " "  # Concatenate all message content

    tokens = tokenizer(full_text, truncation=False, return_tensors="pt")  # Tokenize without truncation
    return tokens["input_ids"].shape[1]  # Token count

# Example usage on the dataset
token_counts = [get_example_token_count(example) for example in training_examples]

# Print statistics
max_length = max(token_counts)
average_length = sum(token_counts) / len(token_counts)

print(f"Max Token Length: {max_length}")
print(f"Average Token Length: {average_length:.2f}")

# Adjust `max_seq_length` accordingly
recommended_max_seq_length = min(max_length, 1024)  # Keep within model limits
print(f"Recommended max_seq_length: {recommended_max_seq_length}")








In [ ]:
print(len(training_examples))

In [ ]:

# torchtune SFT install
# pip install torchtune torch torchvision torchaudio transformers accelerate bitsandbytes peft


In [ ]:
# TRL PPO data format:

{"query": "Scene Graph: {scene_graph} Task: {task_description}",
 "response": "[\"goto(r_10)\", \"goto(r_6)\", \"pickup(o_9)\", \"goto(r_9)\", \"release(o_9)\"]"}
